In [56]:
'数据预处理：增强'
'在做迁移学习的时候，应该保持与(你正在用的)经典网络一致的数据预处理流程，以取得相近的训练结果'

# import
import torch.utils.data.dataloader as dataloader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os


root = 'd:/GitRepos/data/Flower'
datasets_names = ['train', 'valid']

# data transform : preprocessing
data_tranforms = {
    'train' : transforms.Compose([
     transforms.RandomRotation(45)          # 随机旋转，-45~45
    ,transforms.CenterCrop(224)             # 从中心截取固定尺寸224的图片
    ,transforms.RandomHorizontalFlip(0.5)   # 水平翻转，一半一半
    ,transforms.RandomVerticalFlip(0.5)     # 垂直翻转
                                            # 改变亮度、对比度、饱和度、色调 # 虽然pylance报语法错误但是可以正常运行
    ,transforms.ColorJitter(brightness=0.2,contrast=0.1,saturation=0.1,hue=0.1)
    ,transforms.RandomGrayscale(0.5)        # 随机转为灰度图,一半一半
    ,transforms.ToTensor()                  # 转为Tensor
                                            # 归一化
    ,transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ]),

    'valid' : transforms.Compose([
     transforms.Resize(256)                 # 输入图
    ,transforms.CenterCrop(224)
    ,transforms.ToTensor()
    ,transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])
}

'从训练集中分出一部分作为测试集'
# name e.g. daisy
train_dirs = os.listdir(os.path.join(root,datasets_names[0]))
# path e.g. d:/gitrepos/codevcprojects/data/flower/train/daisy
subdirs = {datasets_names[0] : ['/'.join([root,datasets_names[0],train_dir]) for train_dir in train_dirs],
           datasets_names[1] : ['/'.join([root,datasets_names[1],train_dir]) for train_dir in train_dirs]
}
# 创建valid子文件夹
[os.makedirs(path) for path in subdirs[datasets_names[1]] if not os.path.exists(path)]
# 移动文件
[[os.rename(os.path.join(trainpath,img),os.path.join(validpath,img)) 
                for cnt,img in enumerate(os.listdir(trainpath),0)
                if cnt < 20 and len(os.listdir(validpath))==0] 
 for validpath,trainpath in zip(subdirs['valid'],subdirs['train'])]

# 制作数据集
batch_size = 32
image_datasets = {name : datasets.ImageFolder(root=os.path.join(root,name),transform=data_tranforms[name])
        for name in datasets_names
}
image_loader = {
    key : dataloader.DataLoader(
    image_datasets[key],
    batch_size= batch_size,
    shuffle= True)
    for key in image_datasets.keys()
    }
datasets_size = {key : len(image_datasets[key]) for key in image_datasets.keys()}
class_names = image_datasets['train'].classes


{'train': 4217, 'valid': 100}